In [1]:
from googletrans import Translator
translator = Translator(service_urls=['translate.googleapis.com'])

In [2]:
%%time
#th to zh
translator.translate("ฉันไปกินข้าว", src='th', dest='zh-cn').text

Wall time: 212 ms


'我去吃东西了'

In [3]:
%%time
#zh to th
translator.translate("我去吃饭", src='zh-cn',dest='th').text

Wall time: 235 ms


'ฉันกำลังจะไปกิน'

In [4]:
%%time
#zh to th
translator.translate("Have a good day people", src='en',dest='zh-cn').text

Wall time: 257 ms


'祝大家有个美好的一天'

In [11]:
translator.translate("Have a good day people", src='en',dest='zh-cn')

In [2]:
fname = "../data/scb_mt_opus/en-th.merged_stratified.train.csv"
df = pd.read_csv(fname, encoding='utf-8')
df.shape

(3887916, 6)

In [3]:
%%time
x = []
for i in tqdm(df.en.tolist()):
    x.append(translator.translate(i, src='en',dest='zh-cn').text)
    time.sleep(random.randint(1,5))

## Cloud Translation API

In [44]:
import pandas as pd
from tqdm.auto import tqdm

#set up translation client
from google.cloud import translate_v2 as translate
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/mnt/c/charin_projects/NLP-ZH_TH-Project/credentials/charicloud-45aef05e93db.json"

translate_client = translate.Client()

In [49]:
fname = "../data/scb_mt_opus/en-th.merged_stratified.train.csv"
df = pd.read_csv(fname, encoding='utf-8')
df.shape

(3887916, 6)

In [75]:
bs = 100
en_list = df.en.tolist()
th_list = df.th.tolist()
zh_list = []

for i in tqdm(range(len(en_list[:2000])//bs+1)):
    ens = en_list[(i*bs):(i+1)*bs]
    res = translate_client.translate(ens, target_language='zh_cn')
    zh_list+=[i['translatedText'] for i in res]
    if i % (1000/bs) == 0:
        save_df = pd.DataFrame({'en': en_list[:(i+1)*bs], 'th': th_list[:(i+1)*bs], 'zh_translated':zh_list[:(i+1)*bs]})
        print(save_df.shape, len(en_list[:(i+1)*bs]))
        save_df.to_csv(f'{fname[:-4]}_backtranslated.csv', index=False)
save_df = pd.DataFrame({'en': en_list[:(i+1)*bs], 'th': th_list[:(i+1)*bs], 'zh_translated':zh_list[:(i+1)*bs]})
save_df.to_csv(f'{fname[:-4]}_backtranslated.csv', index=False)

(100, 3) 100
(1100, 3) 1100
(2100, 3) 2100



In [76]:
check = pd.read_csv(f'{fname[:-4]}_backtranslated.csv')
check

,en,th,zh_translated
0,The & dolphin; Handbook,คู่มือ & dolphin;,海豚;手册
1,& dolphin; is the default file manager for & k...,& dolphin; คือโปรแกรมจัดการแฟ้มสําหรับ & kde; ...,&amp; 海豚;是 &amp; kde; 的默认文件管理器，设计时以可用性为主要关注点。
2,Filemanager,โปรแกรมจัดการแฟ้ม,文件管理器
3,Introduction,บทนํา,介绍
4,& dolphin; is & kde; 's default file manager. ...,& dolphin; คือโปรแกรมจัดการแฟ้มสําหรับ & kde; ...,&amp; 海豚;是 &amp; kde;的默认文件管理器。它旨在提高用户界面级别的可用性。...
...,...,...,...
2095,Set here the desired custom aspect numerator v...,ตั้งค่าปรับความอิ่มสีได้ที่นี่,在此处设置所需的自定义宽高比分子值。
2096,Set here the desired custom aspect denominator...,ตั้งค่าการชดเชยแสงหลักได้ที่นี่ โดยมีหน่วยเป็น...,在此处设置所需的自定义方面分母值。
2097,Set here the top left selection corner positio...,ตั้งค่ามุมบนซ้ายของการเลือกสําหรับครอบตัดภาพได...,在此处设置用于裁剪的左上角选择角位置。
2098,X:,ตําแหน่ง X:,X：


In [77]:
f'{fname[:-4]}_backtranslated.csv'

'../data/scb_mt_opus/en-th.merged_stratified.train_backtranslated.csv'